# Case Study: Predicting the Cost of Heating Oil Based on Consumption and Past Cost

## Introduction

With the rise of COVI-19, we have observed an increase in local gas prices that may warrant one to switch to gas heating. Since the year 2000, the price of heating oil has more than doubled (Demers, 2022). Although the US government attempts to mitigate the sharp increase in heating oil prices, specially with the incoming cold front that will increase demand of heating oil, outside influences still maintain heating oil prices unstable (Sanicola, 2022). Due to the instability of heating oil prices, a novel machine learning algorithm that predicts future heating oil prices based on past prices will be used. This machine learning algorithm will be developed in the Julia programming language to take full advantage of the programming language's speed and ability to run algorithms directly on the GPU.

## Problem Definition

Current heating oil prices have been very unstable due to the global crises that have emerged. The main goal of the case study is to
1. Normalize the heating oil prices so that the price is independent of the company.
2. Transform the total price to a per gallon bases to create better estimate.
3. Predict Future heating oil prices.

## Exploratory Data Analysis

Before one can begin modeling, it is imperative that the data is explored and the features are examined. In this phase of the product development, one will load the collected data and explore it to observe whether there is any correlation between the features and to engineer certain features that may be missing, such as price per gallon.

We first begin this phase by importing the required libraries

In [9]:
using Plots
using DataFrames
using DelimitedFiles
using Dates
using Flux
using MLUtils

Now that the libraries have been loaded, one can move on to loading the dataset and creating a representative summary of the dataset.

In [10]:
"""
Load CSV data into a DataFrame.
"""
function read_csv(filename::String, delimiter::Char=',')
    data, headers = readdlm(filename, delimiter, header=true)
    df = DataFrame(data, vec(headers))
    return df
end

df = read_csv("../data/heating oil prices.csv")

names(df)

2-element Vector{String}:
 "\ufeffdate"
 "weekly__dollars_per_gallon"

In [11]:
first(df, 10)

Row,\ufeffdate,weekly__dollars_per_gallon
,Any,Any
1,"Oct 01, 1990",1.285
2,"Oct 15, 1990",1.347
3,"Nov 05, 1990",1.355
4,"Nov 19, 1990",1.327
5,"Dec 03, 1990",1.331
6,"Dec 17, 1990",1.28
7,"Jan 07, 1991",1.254
8,"Jan 21, 1991",1.266
9,"Feb 04, 1991",1.24


## Data Processing

First, let us transform the data in to the correct data types.

In [12]:
# for the weekly price of dollars per gallon columns
df[!, :"weekly__dollars_per_gallon"] = convert.(Float32, df[!, :"weekly__dollars_per_gallon"])

# Convert string columns to the date types
df[!, :"\ufeffdate"] = Date.(df[!, :"\ufeffdate"], Dates.DateFormat("u d, yyyy"))
first(df, 10)

Row,\ufeffdate,weekly__dollars_per_gallon
,Date,Float32
1,1990-10-01,1.285
2,1990-10-15,1.347
3,1990-11-05,1.355
4,1990-11-19,1.327
5,1990-12-03,1.331
6,1990-12-17,1.28
7,1991-01-07,1.254
8,1991-01-21,1.266
9,1991-02-04,1.24


One can extract other features from the date column to use as separate inputs or parameters.

In [13]:
df[!,:"years"] = year.(df[!,:"\ufeffdate"])
df[!,:"months"] = month.(df[!,:"\ufeffdate"])
df[!,:"weeks"] = week.(df[!,:"\ufeffdate"])
first(df, 10)

Row,\ufeffdate,weekly__dollars_per_gallon,years,months,weeks
,Date,Float32,Int64,Int64,Int64
1,1990-10-01,1.285,1990,10,40
2,1990-10-15,1.347,1990,10,42
3,1990-11-05,1.355,1990,11,45
4,1990-11-19,1.327,1990,11,47
5,1990-12-03,1.331,1990,12,49
6,1990-12-17,1.28,1990,12,51
7,1991-01-07,1.254,1991,1,2
8,1991-01-21,1.266,1991,1,4
9,1991-02-04,1.24,1991,2,6


Now that the feature engineering aspect of the data has been finished, lets move on to transforming the dataframe into a Flux DataLoader.

In [14]:
Int(nrow(df)*0.75)

522

In [22]:
X_train = Array(first(df[!, [:years,:months,:weeks]], Int(nrow(df)*0.75)))'
X_test = last(df[!, [:years,:months,:weeks]], Int(nrow(df)*0.25))
y_train = Array(first(df[!, :weekly__dollars_per_gallon], Int(nrow(df)*0.75)))'
y_test = last(df[!, :weekly__dollars_per_gallon], Int(nrow(df)*0.25))
train_loader = DataLoader((data=X_train, label=y_train), batchsize=64, shuffle=true);
test_loader = DataLoader((data=X_test, label=y_test), batchsize=64, shuffle=true);


Now that the data has been properly transformed and explored, one should be able to move towards the data modeling phase.

## Data Modeling

Now that the data is ready, let's create a deep learning model that will use the data to learn how to predict heating oil prices.

In [24]:
model = GRU(3 => 1)
optim = Flux.setup(Adam(0.01), model)
losses = []
for epoch in 1:10
    for (x,y) in train_loader
        print(x)
        print("\nThis is the size of x:\n")
        print(size(x))
        loss, grads = Flux.withgradient(model) do m
            # Evaluate model and loss inside gradient context:
            y_hat = m(x)
            crossentropy(y_hat, y)
        end
        update!(optim, model, grads[1])
        push!(losses, loss)
    end
end
optim

[2012 1997 2006 2015 1993 2011 2002 2011 2010 2007 1993 1998 2011 2011 2013 1996 2004 2014 1992 1991 2010 2002 1997 1998 1997 2006 2001 2001 1995 2015 2006 2004 1994 1992 2016 1994 2003 2006 2006 2007 2009 1992 2015 2004 2008 2012 2015 2013 2006 1992 2016 2011 2001 2002 2002 2000 2013 2008 2007 2001 2007 2013 2010 2014; 1 12 2 1 2 10 12 2 3 10 12 1 2 10 11 11 11 12 11 12 10 10 10 3 1 11 3 10 1 1 2 10 12 1 2 1 10 12 2 10 10 3 12 12 12 10 2 2 1 10 2 12 1 11 12 11 12 3 1 10 2 1 2 3; 5 49 6 2 5 40 51 9 9 43 51 4 7 44 48 48 49 1 45 51 42 41 41 10 5 48 10 42 3 5 9 41 49 2 5 5 43 51 8 44 44 12 50 51 50 41 7 6 3 43 6 49 5 45 50 45 51 10 5 43 8 3 7 10]
This is the size of x:
(3, 64)

MethodError: MethodError: no method matching (::Flux.GRUCell{Matrix{Float32}, Matrix{Float32}, Vector{Float32}, Matrix{Float32}})(::Matrix{Float32}, ::Matrix{Int64})
Closest candidates are:
  (::Flux.GRUCell{I, H, V, <:AbstractMatrix{T}})(::Any, ::Union{AbstractVector{T}, AbstractMatrix{T}, OneHotArrays.OneHotArray}) where {I, H, V, T} at C:\Users\wpegu\.julia\packages\Flux\kq9Et\src\layers\recurrent.jl:379